# Documentation

# Sample

In [1]:
from marvin_python_toolbox.common.data import MarvinData

In [2]:
import glob
def read_samples(filename):
    with open(filename, 'r') as fp:
        samples = [line.strip().split() for line in fp.readlines()] 
    positive = [image for image, label in samples if int(label) == 1]
    negative = [image for image, label in samples if int(label) == -1]
    return {'positive': positive, 'negative': negative}
    
train = read_samples('/vagrant/projects/VOCdevkit/VOC2012/ImageSets/Main/aeroplane_train.txt')
val = read_samples('/vagrant/projects/VOCdevkit/VOC2012/ImageSets/Main/aeroplane_val.txt')
    
initial_dataset = ((train, val))

In [3]:
train, val = initial_dataset
dataset = {'train': train, 'val': val}

In [4]:
import os
import numpy as np
import cv2
from keras.layers import Activation, Reshape, Dropout, Dense, Flatten
from keras.layers import AtrousConvolution2D, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D
from keras.models import Sequential
from keras import callbacks, optimizers

Using TensorFlow backend.


In [5]:
def build_model(w=150, h=150, trainable=True):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(w, h, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    if trainable:
        model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.add(Activation('softmax'))

    print(model.summary())
    return model

In [6]:
model = build_model(trainable=True)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [9]:
def generate_samples(image_path, label_path, fnames, w=150, h=150, bs=2):
    indx = 0
    while True:
        for fname in fnames['positive']:
            image = cv2.imread(os.path.join(image_path, fname + '.jpg'))
            image = cv2.resize(image, (w, h))
            image = image[np.newaxis, :, :, (2, 1, 0)]  # [batch=1, w, h, 3]
                
            label = np.array([1])
           
            yield (image, label)  
            
            if indx >= len(fnames['negative']):
                indx = 0
                
            fname = fnames['negative'][indx]
            image = cv2.imread(os.path.join(image_path, fname + '.jpg'))
            image = cv2.resize(image, (w, h))
            image = image[np.newaxis, :, :, (2, 1, 0)]  # [batch=1, w, h, 3]
                
            label = np.array([0])
            
            yield (image, label)
            
            indx += 1

In [10]:
training_data = generate_samples('/vagrant/projects/VOCdevkit/VOC2012/JPEGImages/', 
                                 '/vagrant/projects/VOCdevkit/VOC2012/SegmentationClass/', 
                                  dataset['train'])

validation_data = generate_samples('/vagrant/projects/VOCdevkit/VOC2012/JPEGImages/', 
                                 '/vagrant/projects/VOCdevkit/VOC2012/SegmentationClass/', 
                                  dataset['val'])

model.fit_generator(training_data,
                    steps_per_epoch=2,
                    epochs=1,
                    validation_data=validation_data,
                    validation_steps=1)

Epoch 1/1
2/2 [==============================] - 0s - loss: 7.9712 - acc: 0.5000 - val_loss: 1.1921e-07 - val_acc: 1.0000


In [11]:
from sklearn import metrics as sk_metrics
validation_data = generate_samples('/vagrant/projects/VOCdevkit/VOC2012/JPEGImages/', 
                                   '/vagrant/projects/VOCdevkit/VOC2012/SegmentationClass/', 
                                   dataset['val'])
y_true = []
y_pred = []
for _ in range(3):
    image, label = validation_data.next()
    predicted = model.predict(image)
    y_true.append(label)
    y_pred.append(label)
    
metrics = {}
metrics['model'] = sk_metrics.accuracy_score(y_true, y_pred)

In [14]:
image = cv2.imread('/vagrant/projects/VOCdevkit/VOC2012/JPEGImages/2011_000165.jpg')
image = cv2.resize(image, (150, 150))
image = image[np.newaxis, :, :, (2, 1, 0)]
model.predict(image)

array([[ 1.]], dtype=float32)